In [27]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [28]:
import requests
import numpy as np

In [29]:
leb = 'http://www.basketball-reference.com/players/a/anderal01.html/'
#leb='http://www.basketball-reference.com/players/g/gordoaa01.html'
player_string_key = leb
req = requests.get(player_string_key)
index = req.text.find('div_all_college_stats')
text = req.text[index:]
soup = BeautifulSoup(text, "html.parser")
table = soup.find('table', id = 'all_college_stats')
table

<table class="sortable stats_table" data-cols-to-freeze="1" id="all_college_stats"><caption>College Table</caption>
<colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></col></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class=" over_header center" colspan="3" data-stat=""></th>
<th aria-label="" class=" over_header center" colspan="16" data-stat="header_totals">Totals</th>
<th aria-label="" class=" over_header center" colspan="3" data-stat="header_shooting">Shooting</th>
<th aria-label="" class=" over_header center" colspan="4" data-stat="header_per_g">Per Game</th>
</tr>
<tr>
<th aria-label="If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables." class=" poptip sort_defau

In [30]:
column_headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]

In [31]:
column_headers

['Season',
 'Age',
 'College',
 'G',
 'MP',
 'FG',
 'FGA',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'FG%',
 '3P%',
 'FT%',
 'MP',
 'PTS',
 'TRB',
 'AST']

In [32]:
data_rows = soup.findAll('tr')[2:]  # skip the first two columns

In [33]:
season_data = [[th.getText() for th in data_rows[i].findAll('th')] 
               for i in range(len(data_rows))]

In [34]:
season_data = np.reshape(season_data, -1)

In [35]:
player_data = [[td.getText() for td in data_rows[i].findAll('td')] 
               for i in range(len(data_rows))]

In [36]:
player_data

[['19',
  'MICHIGANST',
  '31',
  '759',
  '67',
  '149',
  '2',
  '5',
  '67',
  '87',
  '',
  '130',
  '51',
  '18',
  '3',
  '',
  '',
  '203',
  '.450',
  '.400',
  '.770',
  '24.5',
  '6.5',
  '4.2',
  '1.6'],
 ['20',
  'MICHIGANST',
  '32',
  '883',
  '90',
  '179',
  '4',
  '13',
  '128',
  '152',
  '',
  '117',
  '104',
  '22',
  '8',
  '',
  '',
  '312',
  '.503',
  '.308',
  '.842',
  '27.6',
  '9.8',
  '3.7',
  '3.3'],
 ['21',
  'MICHIGANST',
  '30',
  '861',
  '78',
  '167',
  '17',
  '48',
  '70',
  '87',
  '',
  '93',
  '96',
  '29',
  '5',
  '',
  '',
  '243',
  '.467',
  '.354',
  '.805',
  '28.7',
  '8.1',
  '3.1',
  '3.2'],
 ['22',
  'MICHIGANST',
  '33',
  '877',
  '149',
  '268',
  '25',
  '65',
  '114',
  '130',
  '',
  '184',
  '56',
  '32',
  '7',
  '',
  '',
  '437',
  '.556',
  '.385',
  '.877',
  '26.6',
  '13.2',
  '5.6',
  '1.7'],
 ['',
  '',
  '126',
  '3380',
  '384',
  '763',
  '48',
  '131',
  '379',
  '456',
  '',
  '524',
  '307',
  '101',
  '23',
  ''

In [37]:
df = pd.DataFrame(player_data, columns=column_headers[1:])

In [38]:
df["Season"]=season_data

In [39]:
df.head()

,Age,College,G,MP,FG,FGA,3P,3PA,FT,FTA,...,PF,PTS,FG%,3P%,FT%,MP,PTS,TRB,AST,Season
0,19,MICHIGANST,31,759,67,149,2,5,67,87,...,,203,.450,.400,.770,24.5,6.5,4.2,1.6,2001-02
1,20,MICHIGANST,32,883,90,179,4,13,128,152,...,,312,.503,.308,.842,27.6,9.8,3.7,3.3,2002-03
2,21,MICHIGANST,30,861,78,167,17,48,70,87,...,,243,.467,.354,.805,28.7,8.1,3.1,3.2,2003-04
3,22,MICHIGANST,33,877,149,268,25,65,114,130,...,,437,.556,.385,.877,26.6,13.2,5.6,1.7,2004-05
4,,,126,3380,384,763,48,131,379,456,...,,1195,.503,.366,.831,26.8,9.5,4.2,2.4,Career


In [40]:
def college_data_scraper(url):
    ## scrape the college table
    req = requests.get(url)
    index = req.text.find('div_all_college_stats')
    text = req.text[index:]
    soup = BeautifulSoup(text, "html.parser")
    table = soup.find('table', id = 'all_college_stats')
    
    ## extract column headers
    column_headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    
    ## extract contents from the table
    data_rows = soup.findAll('tr')[2:]  # skip the first two columns
    player_data = [[td.getText() for td in data_rows[i].findAll('td')] 
               for i in range(len(data_rows))]
    
    ## extract the first column (season)
    season_data = [[th.getText() for th in data_rows[i].findAll('th')] 
               for i in range(len(data_rows))]
    season_data = np.reshape(season_data, -1)
    
    ## combine the columns and data as a data frame
    df = pd.DataFrame(player_data, columns=column_headers[1:])
    df["Season"] = season_data
    return(df)

In [41]:
test = college_data_scraper("http://www.basketball-reference.com/players/b/bennean01.html")
test.head()

,Age,College,G,MP,FG,FGA,3P,3PA,FT,FTA,...,PF,PTS,FG%,3P%,FT%,MP,PTS,TRB,AST,Season
0,19,UNLV,35,950,202,379,36,96,124,177,...,80,564,.533,.375,.701,27.1,16.1,8.1,1.0,2012-13
1,,,35,950,202,379,36,96,124,177,...,80,564,.533,.375,.701,27.1,16.1,8.1,1.0,Career


In [42]:
base = 'http://www.basketball-reference.com/players/'
player='b/bennean01'
ext='.html'
base+player+ext

'http://www.basketball-reference.com/players/b/bennean01.html'